# Python для сбора и обработки данных

Автор: Валентин Бирюков

# Задача - изучить "конкурентов"

История: 

    Вот работаем мы в проекте "Крутые вещи и не только" (которы продает вещи разных брендов, почему бы и нет). И нам пришли с заказом - Хотят, мол, посмотреть что там у конкрурентов - сколько позиций, каких товаров, какие скидки, какие бренды.
    
    Мы конечно готовы все проанализировать... но данные-то, данные где брать? не пойдем же мы к конкурентам с вопросами? Придется собирать самим. С нуля...

# Подготовка

Нам понадобится что то парсить. Давайте возмем www.lyst.com.au - строго говоря разницы нет, можно взять еще какой нибудь популярный магазин, но этот тоже удобный.

Посмотрим что мы хотим получить с него в целом - ага, есть разные категории, есть цена до скидки, после скидки, сама картиночка, название и бренд. звучит неплохо, попробуем начать с этого.

In [30]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import re

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

headers={'User-Agent':user_agent,} 

def make_soup(url):
    print(url)
    request= urllib.request.Request(url, None,headers) #The assembled request
    thepage = urllib.request.urlopen(request)
    soupdata = BeautifulSoup(thepage, "html.parser")
    return soupdata

# Научимся парсить один кусок данных

Нужно: выцепить все полезное что можно из одного `product-card` на странице.

In [31]:
def get_entity(card, tag='div', prod_class={}, item_id=0):
    try:
        return card.findAll(tag, prod_class)[item_id].get_text().strip()
    except:
        return ''
    

In [32]:
all_products = []
base_url = 'https://www.lyst.com.au{link}?page={num}'


In [33]:
import json
soup = make_soup(base_url.format(link='/shop/wedge-boots/', num=1))

navigator = json.loads(soup.findAll('script', {'data-hypernova-key': 'MainNavigation'})[0].get_text()[4:-3])

https://www.lyst.com.au/shop/wedge-boots/?page=1


In [7]:
for gender in navigator['data'].values():
    for category in gender['menus']:
        for item_type in category['menus']:
            for tab in item_type['links']:
                print(gender['display_name'], category['display_name'], item_type['display_name'], tab['text'])
                for page_id in range(5):
                    soup = make_soup(base_url.format(link=tab['url'], num=page_id))
                    cards = soup.findAll('div', {'class': 'product-card'})
                    print('Collecting {} items'.format(len(all_products)))
                    for product in cards:
                        parsed_item = dict(
                            disigner = get_entity(
                                product, prod_class={'class': 'product-card__designer'}
                            ),
                            name = get_entity(
                                product, prod_class={'class': 'product-card__short-description-name'}
                            ),
                            price = re.sub('[^0-9]+', '', get_entity(
                                product, 'del', prod_class={'class': 'product-card__price'})
                            ) or re.sub('[^0-9]+', '', get_entity(
                                product, 'span', prod_class={'class': 'product-card__price'})
                            ),
                            price_sale = re.sub('[^0-9]+', '', get_entity(
                                product, 'span', {'class': 'product-card__price product-card__price--sale'})
                            ),
                            ower = get_entity(
                                product, 'span', item_id=-1
                            ),
                            image = product.findAll('img')[0].get('src') or product.findAll('img')[0].get('image-src'),
                            gender = gender['display_name'],
                            category = category['display_name'],
                            item_type = item_type['display_name'],
                            tab = tab['text']
                        )
                        all_products.append(parsed_item)

women Clothing Beachwear Cover-ups and kaftans
https://www.lyst.com.au/shop/kaftans/?page=0
https://www.lyst.com.au/shop/kaftans/?page=1
https://www.lyst.com.au/shop/kaftans/?page=2
https://www.lyst.com.au/shop/kaftans/?page=3
https://www.lyst.com.au/shop/kaftans/?page=4
women Clothing Beachwear Beach towels
https://www.lyst.com.au/shop/towels/?page=0
https://www.lyst.com.au/shop/towels/?page=1
https://www.lyst.com.au/shop/towels/?page=2
https://www.lyst.com.au/shop/towels/?page=3
https://www.lyst.com.au/shop/towels/?page=4
women Clothing Beachwear Sarongs
https://www.lyst.com.au/shop/sarongs/?page=0


KeyboardInterrupt: 

# Парсим много страниц выдачи

Запустим парсер для кучи разных страниц

# Распарсим заголовок - в нем лежат категории и ссылки

# Подготовмися к сбору кучи страниц

# Думаем над ускорением

In [9]:
links = []
for gender in navigator['data'].values():
    for category in gender['menus']:
        for item_type in category['menus']:
            for tab in item_type['links']:
                links.append(tab['url'])
print(len(links))

355


In [5]:
from multiprocessing.pool import Pool, ThreadPool
from time import sleep

In [9]:
def dummy(x):
    sleep(1)
    return x*x

a = list(range(10))

In [25]:
%%time
with ThreadPool(10) as pool:
    test_1 = pool.map(dummy, a)


CPU times: user 3.69 ms, sys: 3.39 ms, total: 7.08 ms
Wall time: 1.04 s


In [11]:
%%time
test_2 = list(map(dummy, a))

CPU times: user 1.22 ms, sys: 1.7 ms, total: 2.92 ms
Wall time: 10 s


In [26]:
assert test_1 == test_2

# Причесываем код под мультипоточность

In [36]:
links_parmas = []
for gender in navigator['data'].values():
    for category in gender['menus']:
        for item_type in category['menus']:
            for tab in item_type['links']:
                links_parmas.append((
                    tab['url'], gender['display_name'],
                    category['display_name'],
                    item_type['display_name'], tab['text']
                ))
print(len(links_parmas))

355


In [52]:
def parse_category(params):
    url_part, gender, category, item_type, tab = params
    all_products = []
    for page_id in range(5):
        try: 
            soup = make_soup(base_url.format(link=url_part, num=page_id))
        except:
            print("МЫ УПАЛИ ЗДЕСЬ")
            continue
        cards = soup.findAll('div', {'class': 'product-card'})
        for product in cards:
            parsed_item = dict(
                disigner = get_entity(
                    product, prod_class={'class': 'product-card__designer'}
                ),
                name = get_entity(
                    product, prod_class={'class': 'product-card__short-description-name'}
                ),
                price = re.sub('[^0-9]+', '', get_entity(
                    product, 'del', prod_class={'class': 'product-card__price'})
                ) or re.sub('[^0-9]+', '', get_entity(
                    product, 'span', prod_class={'class': 'product-card__price'})
                ),
                price_sale = re.sub('[^0-9]+', '', get_entity(
                    product, 'span', {'class': 'product-card__price product-card__price--sale'})
                ),
                ower = get_entity(
                    product, 'span', item_id=-1
                ),
                image = product.findAll('img')[0].get('src') or product.findAll('img')[0].get('image-src'),
                gender = gender,
                category = category,
                item_type = item_type,
                tab = tab
            )
            all_products.append(parsed_item)
    return all_products

In [55]:
355*5*48/(25*5*48)*60/60

14.2

In [56]:
%%time
data = []
with ThreadPool(20) as pool:
    for result in pool.map(parse_category, links_parmas):
        data.extend(result)

https://www.lyst.com.au/shop/towels/?page=0
https://www.lyst.com.au/shop/beachwear/?page=0
https://www.lyst.com.au/shop/parka-coats/?page=0
https://www.lyst.com.au/shop/casual-dresses/?page=0
https://www.lyst.com.au/shop/dresses/?page=0https://www.lyst.com.au/shop/hosiery/?page=0

https://www.lyst.com.au/shop/formal-jackets/?page=0https://www.lyst.com.au/shop/jackets/sale/?page=0

https://www.lyst.com.au/shop/skinny-jeans/?page=0https://www.lyst.com.au/shop/full-length-jumpsuits/?page=0

https://www.lyst.com.au/shop/turtlenecks/?page=0
https://www.lyst.com.au/shop/knitwear/sale/?page=0
https://www.lyst.com.au/shop/camisoles/?page=0https://www.lyst.com.au/shop/robes/?page=0
https://www.lyst.com.au/shop/skinny-pants/?page=0
https://www.lyst.com.au/shop/full-length-pants/?page=0
https://www.lyst.com.au/shop/knee-length-shorts/?page=0
https://www.lyst.com.au/shop/shorts/sale/?page=0

https://www.lyst.com.au/shop/skirts/?page=0
https://www.lyst.com.au/shop/suits/sale/?page=0
https://www.lys

https://www.lyst.com.au/shop/fur-coats/?page=2
https://www.lyst.com.au/shop/sweatshirts/?page=3
https://www.lyst.com.au/shop/flared-jeans/?page=3
https://www.lyst.com.au/shop/pyjamas/?page=3https://www.lyst.com.au/shop/wide-leg-pants/?page=3

https://www.lyst.com.au/shop/dresses/sale/?page=4
https://www.lyst.com.au/shop/cargo-pants/?page=3
https://www.lyst.com.au/shop/straight-jeans/?page=3
https://www.lyst.com.au/shop/ponchos/?page=3
https://www.lyst.com.au/shop/formal-shorts/?page=4
https://www.lyst.com.au/shop/parka-jackets/?page=3
https://www.lyst.com.au/shop/maxi-dresses/?page=4https://www.lyst.com.au/shop/bodysuits/?page=3

https://www.lyst.com.au/shop/bras/?page=4https://www.lyst.com.au/shop/jumpsuits/?page=3

https://www.lyst.com.au/shop/hosiery/sale/?page=3
https://www.lyst.com.au/shop/mid-length-skirts/?page=4
https://www.lyst.com.au/shop/skirts/sale/?page=3
https://www.lyst.com.au/shop/one-piece-swimsuits/?page=4
https://www.lyst.com.au/shop/fur-coats/?page=3
https://www.lys

https://www.lyst.com.au/shop/denim-shorts/?page=2https://www.lyst.com.au/shop/corsetry/?page=1

https://www.lyst.com.au/shop/pants/sale/?page=1
https://www.lyst.com.au/shop/mini-skirts/?page=2
https://www.lyst.com.au/shop/bootcut-jeans/?page=2
https://www.lyst.com.au/shop/jeans/sale/?page=2
https://www.lyst.com.au/shop/sarongs/?page=2
https://www.lyst.com.au/shop/straight-leg-pants/?page=2
https://www.lyst.com.au/shop/gowns/?page=2https://www.lyst.com.au/shop/lingerie/sale/?page=1

https://www.lyst.com.au/shop/socks/?page=3
https://www.lyst.com.au/shop/denim-jackets/?page=1https://www.lyst.com.au/shop/sleeveless-sweaters/?page=2
https://www.lyst.com.au/shop/sweatpants/?page=2

https://www.lyst.com.au/shop/coats/?page=1
https://www.lyst.com.au/shop/nightwear/sale/?page=3
https://www.lyst.com.au/shop/denim-shorts/?page=3
https://www.lyst.com.au/shop/fur-jackets/?page=2
https://www.lyst.com.au/shop/short-coats/?page=2
https://www.lyst.com.au/shop/sweaters/?page=2
https://www.lyst.com.au/s

https://www.lyst.com.au/shop/scarves/?page=0
https://www.lyst.com.au/shop/ankle-boots/?page=1
https://www.lyst.com.au/shop/sweats/?page=1https://www.lyst.com.au/shop/blouses/?page=1

https://www.lyst.com.au/shop/umbrellas/sale/?page=0https://www.lyst.com.au/shop/backpacks/?page=0
https://www.lyst.com.au/shop/beach-and-straw-bags/sale/?page=0

https://www.lyst.com.au/shop/boots/sale/?page=1https://www.lyst.com.au/shop/coats/sale/?page=4

https://www.lyst.com.au/shop/bucket-bags/?page=0
https://www.lyst.com.au/shop/over-the-knee-boots/?page=1
https://www.lyst.com.au/shop/lace-ups/?page=1
https://www.lyst.com.au/shop/sandal-heels/?page=1https://www.lyst.com.au/shop/gloves/sale/?page=1

https://www.lyst.com.au/shop/belts/?page=1
https://www.lyst.com.au/shop/crossbody-bags/sale/?page=0https://www.lyst.com.au/shop/luggage/?page=0
https://www.lyst.com.au/shop/heels/sale/?page=1

https://www.lyst.com.au/shop/high-heels/?page=1
https://www.lyst.com.au/shop/satchels/sale/?page=0
https://www.lyst

https://www.lyst.com.au/shop/sleeveless-tops/?page=0
https://www.lyst.com.au/shop/wallets/?page=2
https://www.lyst.com.au/shop/belts/sale/?page=4
https://www.lyst.com.au/shop/hobo-bags/?page=3
https://www.lyst.com.au/shop/flip-flops/?page=4
https://www.lyst.com.au/shop/backpacks/sale/?page=3https://www.lyst.com.au/shop/shoulder-bags/?page=3

https://www.lyst.com.au/shop/wedge-shoes/?page=4https://www.lyst.com.au/shop/luggage/sale/?page=3
https://www.lyst.com.au/shop/scarves/sale/?page=4

https://www.lyst.com.au/shop/wellington-boots/?page=4
https://www.lyst.com.au/shop/bucket-bags/sale/?page=3
https://www.lyst.com.au/shop/belt-bags/sale/?page=0
https://www.lyst.com.au/shop/hair/sale/?page=0
https://www.lyst.com.au/shop/flat-sandals/?page=0
https://www.lyst.com.au/shop/mid-calf-boots/?page=4
https://www.lyst.com.au/shop/wedge-sandals/?page=4
https://www.lyst.com.au/shop/shirts/?page=1
https://www.lyst.com.au/shop/wallets/?page=3
https://www.lyst.com.au/shop/totes/sale/?page=2
https://ww

...

https://www.lyst.com.au/shop/mens-casual-pants/?page=0
https://www.lyst.com.au/shop/mens-formal-shirts/?page=0
https://www.lyst.com.au/shop/mens-formal-shorts/?page=0
https://www.lyst.com.au/shop/mens-2-piece-suits/?page=0
https://www.lyst.com.au/shop/mens-beachwear/?page=1
https://www.lyst.com.au/shop/mens-tracksuits/?page=0
https://www.lyst.com.au/shop/mens-tapered-jeans/?page=1
https://www.lyst.com.au/shop/mens-short-coats/?page=1
https://www.lyst.com.au/shop/mens-sweats/sale/?page=0
https://www.lyst.com.au/shop/mens-formal-jackets/?page=1
https://www.lyst.com.au/shop/mens-t-shirts/?page=0https://www.lyst.com.au/shop/brooches/?page=3

https://www.lyst.com.au/shop/brooches/sale/?page=3
https://www.lyst.com.au/shop/mens-slim-jeans/?page=1
https://www.lyst.com.au/shop/mens-undershirts/?page=0
https://www.lyst.com.au/shop/mens-knitwear/sale/?page=1
https://www.lyst.com.au/shop/mens-turtlenecks/?page=1https://www.lyst.com.au/shop/mens-desert-boots/?page=0

https://www.lyst.com.au/shop/me

https://www.lyst.com.au/shop/earrings/sale/?page=0
https://www.lyst.com.au/shop/mens-formal-pants/?page=3https://www.lyst.com.au/shop/mens-shirts/?page=3
https://www.lyst.com.au/shop/mens-sweatshorts/?page=3

https://www.lyst.com.au/shop/mens-cardigans/?page=3
https://www.lyst.com.au/shop/mens-coats/?page=4
https://www.lyst.com.au/shop/mens-oxfords/?page=2
https://www.lyst.com.au/shop/mens-casual-jackets/?page=3
https://www.lyst.com.au/shop/mens-jeans/?page=4
https://www.lyst.com.au/shop/mens-robes/?page=4
https://www.lyst.com.au/shop/mens-sleeveless-t-shirts/?page=3
https://www.lyst.com.au/shop/mens-straight-jeans/?page=4
https://www.lyst.com.au/shop/rings/sale/?page=4
https://www.lyst.com.au/shop/mens-t-shirts/sale/?page=3
https://www.lyst.com.au/shop/mens-evening-suits/?page=3https://www.lyst.com.au/shop/mens-trench-coats/?page=0

https://www.lyst.com.au/shop/mens-sandals/?page=1
https://www.lyst.com.au/shop/mens-formal-boots/?page=3
https://www.lyst.com.au/shop/mens-underwear/?page

https://www.lyst.com.au/shop/mens-long-sleeve-t-shirts/?page=0
https://www.lyst.com.au/shop/mens-boxers/?page=0
https://www.lyst.com.au/shop/mens-waistcoats/?page=2
https://www.lyst.com.au/shop/mens-pants/sale/?page=1
https://www.lyst.com.au/shop/mens-parka-coats/?page=2
https://www.lyst.com.au/shop/necklaces/?page=3https://www.lyst.com.au/shop/mens-casual-shorts/?page=1
https://www.lyst.com.au/shop/mens-relaxed-jeans/?page=1

https://www.lyst.com.au/shop/mens-shorts/sale/?page=2
https://www.lyst.com.au/shop/mens-boots/sale/?page=0https://www.lyst.com.au/shop/mens-v-neck/?page=2

https://www.lyst.com.au/shop/mens-towels/?page=2
https://www.lyst.com.au/shop/mens-lace-ups/sale/?page=0
https://www.lyst.com.au/shop/mens-suits/sale/?page=1
https://www.lyst.com.au/shop/mens-sandals/sale/?page=4
https://www.lyst.com.au/shop/mens-nightwear/?page=2
https://www.lyst.com.au/shop/mens-hoodies/?page=1
https://www.lyst.com.au/shop/mens-casual-boots/?page=1
https://www.lyst.com.au/shop/mens-long-slee

https://www.lyst.com.au/shop/mens-hi-top-sneakers/?page=0
https://www.lyst.com.au/shop/mens-3-piece-suits/?page=4
https://www.lyst.com.au/shop/mens-boardshorts/?page=4
https://www.lyst.com.au/shop/mens-cargo-shorts/?page=4
https://www.lyst.com.au/shop/mens-sweats/?page=3
https://www.lyst.com.au/shop/mens-jackets/sale/?page=4
https://www.lyst.com.au/shop/mens-skinny-jeans/?page=4https://www.lyst.com.au/shop/mens-knitwear/?page=3

https://www.lyst.com.au/shop/mens-casual-shirts/?page=4https://www.lyst.com.au/shop/mens-flip-flops/?page=2

https://www.lyst.com.au/shop/mens-polo-shirts/?page=3
https://www.lyst.com.au/shop/mens-belts/sale/?page=0
https://www.lyst.com.au/shop/mens-socks/?page=3
https://www.lyst.com.au/shop/mens-derbies/?page=3
https://www.lyst.com.au/shop/mens-sweatshirts/?page=4
https://www.lyst.com.au/shop/mens-wellington-boots/?page=3
https://www.lyst.com.au/shop/mens-hats/?page=0
https://www.lyst.com.au/shop/mens-sunglasses/sale/?page=0
https://www.lyst.com.au/shop/mens-w

https://www.lyst.com.au/shop/mens-messenger/sale/?page=1
https://www.lyst.com.au/shop/mens-cufflinks/sale/?page=2https://www.lyst.com.au/shop/mens-tote-bags/?page=1

https://www.lyst.com.au/shop/mens-backpacks/?page=2
https://www.lyst.com.au/shop/mens-scarves/?page=2
https://www.lyst.com.au/shop/mens-briefcases/sale/?page=2
https://www.lyst.com.au/shop/mens-ties/sale/?page=2
https://www.lyst.com.au/shop/mens-sneakers/?page=2
https://www.lyst.com.au/shop/mens-slippers/?page=1
https://www.lyst.com.au/shop/mens-cufflinks/sale/?page=3https://www.lyst.com.au/shop/mens-necklaces/sale/?page=2

https://www.lyst.com.au/shop/mens-holdalls/?page=2
https://www.lyst.com.au/shop/mens-messenger/sale/?page=2
https://www.lyst.com.au/shop/mens-backpacks/?page=3
https://www.lyst.com.au/shop/mens-tote-bags/?page=2
https://www.lyst.com.au/shop/mens-briefcases/sale/?page=3
https://www.lyst.com.au/shop/mens-scarves/?page=3
https://www.lyst.com.au/shop/mens-ties/sale/?page=3
https://www.lyst.com.au/shop/mens-

In [57]:
len(data)

84359

# Любуемся результатом

In [58]:
import pandas as pd

In [59]:
data = pd.DataFrame(data)

In [61]:
data.sort_values(by = 'price', ascending=False)

,category,disigner,gender,image,item_type,name,ower,price,price_sale,tab
32544,Shoes,Chloé,women,https://cdna.lystit.com/200/250/tr/photos/myth...,Heels,Camille Suede Platform Sandals - Pink,Mytheresa,999,699,Wedge sandals
33842,Shoes,Nike,women,https://cdna.lystit.com/200/250/tr/photos/farf...,Trainers,Air 1 Retro High Og Obsidian/university Blue,Farfetch,999,,Low-top trainers
33794,Shoes,Nike,women,https://cdna.lystit.com/200/250/tr/photos/farf...,Trainers,Air 1 Retro High Og Obsidian/university Blue,Farfetch,999,,Low-top trainers
16639,Clothing,Michael Kors,women,https://cdna.lystit.com/200/250/tr/photos/mich...,Trousers,Selma Leather Skinny Trousers - Black,Michael Kors,999,289,Skinny trousers
18654,Clothing,Michael Kors,women,https://cdna.lystit.com/200/250/tr/photos/mich...,Trousers,Selma Leather Skinny Trousers - Black,Michael Kors,999,289,Sale Trousers
71120,Shoes,Ferragamo,men,https://cdna.lystit.com/200/250/tr/photos/thel...,Slip-ons,Green Suede Gancio Bit Loafers Size 44.5,The Luxury Closet,999,265,Loafers
72049,Shoes,Ferragamo,men,https://cdna.lystit.com/200/250/tr/photos/thel...,Slip-ons,Green Suede Gancio Bit Loafers Size 44.5,The Luxury Closet,999,265,All Slip-ons
34035,Shoes,Nike,women,https://cdna.lystit.com/200/250/tr/photos/farf...,Trainers,Air 1 Retro High Og Obsidian/university Blue,Farfetch,999,,All Trainers
71072,Shoes,Ferragamo,men,https://cdna.lystit.com/200/250/tr/photos/thel...,Slip-ons,Green Suede Gancio Bit Loafers Size 44.5,The Luxury Closet,999,265,Loafers
72279,Shoes,Ferragamo,men,https://cdna.lystit.com/200/250/tr/photos/thel...,Slip-ons,Green Suede Gancio Bit Loafers Size 44.5,The Luxury Closet,999,265,Sale Slip-ons
